In [7]:
import tensorflow as tf
sess = tf.Session()
#tf.enable_eager_execution()

In [42]:
#split data
import pandas as pd
df = pd.read_csv("../data/train_data_reformatted.csv")
train, validate = np.split(df.sample(frac=1), [int(.8*len(df))])
print(train.shape)
print(validate.shape)
train.to_csv('../data/processed_data/small_train.csv', header=False, index=False)
validate.to_csv('../data/processed_data/small_validate.csv', header=False, index=False)

(3199, 151)
(800, 151)


In [43]:
CSV_COLUMNS = ['signal_observation_' + str(i) for i in range(150)] + ['time_to_failure']
print(CSV_COLUMNS[-3:])
LABEL_COLUMN = 'time_to_failure'
DEFAULTS = [[0.0] for i in range(150)] + [[0.0]]

def read_dataset(filename, mode, batch_size = 512):
    def decode_csv(value_column):
        columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
        features = dict(zip(CSV_COLUMNS, columns))
        label = features.pop(LABEL_COLUMN)
        return features, label

    # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
    filenames_dataset = tf.data.Dataset.list_files(filename)
    # Read lines from text files
    textlines_dataset = filenames_dataset.flat_map(tf.data.TextLineDataset)
    # Parse text lines as comma-separated values (CSV)
    dataset = textlines_dataset.map(decode_csv)

    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)
    else:
        num_epochs = 1 # end-of-input after this

    dataset = dataset.repeat(num_epochs).batch(batch_size)

    return dataset

['signal_observation_148', 'signal_observation_149', 'time_to_failure']


In [44]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column(i) for i in CSV_COLUMNS[:-1]
]
INPUT_COLUMNS[-3:]

[_NumericColumn(key='signal_observation_147', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='signal_observation_148', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='signal_observation_149', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [45]:
# iterator = read_dataset("../data/train_data_reformatted.csv",mode=tf.estimator.ModeKeys.TRAIN, batch_size=10).make_one_shot_iterator()
# print(sess.run(iterator.get_next()))


In [46]:


# # Test if the data pipeline works using a simple estimator (predictibly, it performs quite terribly)
# estimator = tf.estimator.LinearRegressor(
#     feature_columns=INPUT_COLUMNS)

# estimator.train(input_fn=lambda: read_dataset("../data/train_data_reformatted.csv",mode=tf.estimator.ModeKeys.TRAIN, batch_size=500))


In [57]:
import numpy as np
from tensorflow.contrib import layers

def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    features = tf.feature_column.input_layer(
            features=features,
            feature_columns=INPUT_COLUMNS)
    if mode != tf.estimator.ModeKeys.PREDICT:
        labels = tf.reshape(labels, [-1, 1])
    input_layer = tf.reshape(features, [-1, 1, 150, 1])
    
    # Convolutional Layer #1
    # Note, these 2D conv layers are being hijacked to do 1D convolutions
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[1, 16],
      padding="same",
      activation=tf.nn.relu)
    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[1, 3], strides=3)
    
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[32, 16],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[1, 5], strides=5)
    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1,640])
    dense = tf.layers.dense(inputs=pool2_flat, units=20, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)


    predictions = tf.layers.dense(inputs=dropout, units=1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return predictions
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.mean_squared_error(labels, predictions)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss)


In [58]:
# signal_regressor = tf.estimator.Estimator(
#     model_fn=cnn_model_fn, model_dir="./tmp/lanl_convnet_model")
# signal_regressor

In [59]:

#signal_regressor.train(input_fn=lambda: read_dataset("../data/train_data_reformatted.csv", mode=tf.estimator.ModeKeys.TRAIN, batch_size=256*2))


In [60]:
# for i in signal_regressor.predict(input_fn=lambda: read_dataset("../data/train_data_reformatted.csv", mode=tf.estimator.ModeKeys.PREDICT, batch_size=1)):
#     print(i)


In [61]:
# iterator = read_dataset("../data/train_data_reformatted.csv",mode=tf.estimator.ModeKeys.PREDICT, batch_size=100).make_one_shot_iterator()

In [62]:
# features, labels = iterator.get_next()
# a = zip(cnn_model_fn(features, labels, tf.estimator.ModeKeys.PREDICT), labels)
# mse = 0
# k = 0
# for i, j in a:
#     print(i.numpy(), j.numpy())
#     mse += (i.numpy()[0] - j.numpy())**2
#     k = k + 1
# print(mse/ k)

In [73]:
def train_and_evaluate(output_dir):
    TRAIN_STEPS = 10
    EVAL_INTERVAL = 300
    run_config = tf.estimator.RunConfig(save_checkpoints_secs = EVAL_INTERVAL,
                                      keep_checkpoint_max = 3)
    signal_regressor = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir=output_dir)
    
    

    
    train_spec = tf.estimator.TrainSpec(
                       input_fn = lambda: read_dataset("../data/processed_data/small_train.csv", mode=tf.estimator.ModeKeys.TRAIN, batch_size=256),
                       max_steps = TRAIN_STEPS)
    
    eval_spec = tf.estimator.EvalSpec(
                       input_fn = lambda: read_dataset("../data/processed_data/small_validate.csv", mode=tf.estimator.ModeKeys.EVAL, batch_size=256))
    tf.estimator.train_and_evaluate(signal_regressor, train_spec, eval_spec)

In [74]:
train_and_evaluate("./tmp/lanl_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tmp/lanl_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c2fa2a320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tenso

In [1]:
import model

In [3]:
import tensorflow as tf
signal_regressor = tf.estimator.Estimator(
        model_fn=model.cnn_model_fn, model_dir='../single_large_gpu_model/single_large_gpu_model/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../single_large_gpu_model/single_large_gpu_model/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb241dbc88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
for i in signal_regressor.predict(input_fn = lambda: model.read_dataset("test", mode=tf.estimator.ModeKeys.PREDICT, batch_size=64)):
    print(i)

NameError: name 'BATCH_SIZE' is not defined